In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D


In [ ]:
#generators
train_ds=keras.utils.image_dataset_from_directory(
    directory='/content/train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)

)

valid_ds=keras.utils.image_dataset_from_directory(
    directory='/content/test',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)

)

In [ ]:
#Normalize
def process(image,label):
  image=tensorflow.cast(image/255. ,tensorflow.float32)
  return image,label

train_ds=train_ds.map(process)
valid_ds=valid_ds.map(process)

In [ ]:
model=Sequential()

In [ ]:
#Model
model.add(Conv2D(64,kernel_size=(3,3),activation='relu',input_shape=(256,256,3),padding='valid'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(train_ds,validation_data=valid_ds,epochs=5)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

In [ ]:
import cv2
import matplotlib.pyplot as plt
test_img=cv2.imread('/content/angry.jpeg')
plt.imshow(test_img)

In [ ]:
test_img.shape

In [ ]:
test_img=cv2.resize(test_img,(256,256))
plt.imshow(test_img)


In [ ]:
test_input=test_img.reshape((1,256,256,3)) ##1 for input one image at one and other is shape

In [ ]:
import numpy as np
model.predict((test_input))

## Post Data Augumentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,            # Rescale pixel values to [0, 1]
    rotation_range=40,         # Randomly rotate images by up to 40 degrees
    width_shift_range=0.2,     # Randomly translate images horizontally by 20%
    height_shift_range=0.2,    # Randomly translate images vertically by 20%
    shear_range=0.2,           # Randomly shear images
    zoom_range=0.2,            # Randomly zoom into images
    horizontal_flip=True,      # Randomly flip images horizontally
    fill_mode='nearest'        # Fill pixels after augmentations
)

# For validation data, you generally do not augment but just rescale
validation_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/content/train',        # This is the target directory
    target_size=(256, 256),     # Resize images to 150x150
    batch_size=32,
    class_mode='binary',
    color_mode='rgb' # Since it's binary classification
)

validation_generator = validation_datagen.flow_from_directory(
    '/content/test',   # This is the target directory for validation data
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    color_mode='rgb'
)


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=5,       # Number of batches to draw from the generator per epoch
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50        # Number of batches to draw from the generator for validation
)


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the image
img_path = '/content/happy.jpeg'  # Replace with the path to your image
img = image.load_img(img_path, target_size=(256, 256))  # Resize to the same size as the training images

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Normalize the pixel values (rescale)
img_array = img_array / 255.0

# Add a batch dimension since the model expects batches of images
img_array = np.expand_dims(img_array, axis=0)

In [ ]:
# Make a prediction
prediction = model.predict(img_array)

# Since it's a binary classification problem, interpret the result
if prediction[0] > 0.5:
    print(f"The image is predicted to be a happy with a probability of {prediction[0][0]:.2f}")
else:
    print(f"The image is predicted to be a angry with a probability of {1 - prediction[0][0]:.2f}")
